## Model Test
####  Training datasets used here are in .zip compressed version

In [254]:
import pandas as pd
import ast
import numpy as np
import random
import re
from collections import Counter
import networkx as nx
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import ast
import csv
import pickle

## Model 1: Network Based Model

In [303]:
metadata_df = pd.read_csv('../dataset/network_metadata.zip', compression='zip', header=0, sep=',')
# metadata_df['cast'] = metadata_df['cast'].apply(lambda x: ast.literal_eval(x))

In [499]:
metadata_df.sample(3)

,index,belongs_to_collection,budget,genres,id,imdbId,original_language,overview,popularity,release_date,...,vote_count,cast,director,writer,producer,executive producer,score,wikiId,Summary,release_year
8849,11799,None,0,"['Mystery', 'Horror', 'Thriller']",16764,367153.0,en,A historic Hollywood hotel houses a supernatur...,4.710011,2004-11-12,...,37.0,"['Brent Roam', 'Angela Bettis', 'Rance Howard'...",['Tobe Hooper'],[],"['Terence S. Potter', 'Jacqueline Quella', 'To...","['Ronnie Truss', 'Mark Wooding', 'Ryan Carroll...",[],7960697,"In a small apartment complex, somebody in a sk...",2004.0
19755,24187,None,0,['Horror'],87852,85937.0,en,A guy kills his wife because she's a bad cook.,0.084197,1983-08-31,...,19.0,"['Jackie Vernon', 'Loren Schein', 'Al Troupe',...",['Wayne Berwick'],['Thomas Singer'],"['Craig Muckler', 'Thomas Singer']",[],['Leif Horvath'],26938419,Donald is a construction worker with a big pr...,1983.0
567,788,None,0,"['Drama', 'Family', 'Adventure']",21352,109120.0,en,The true story of how a marine seal named Andr...,3.139316,1994-08-17,...,31.0,"['Tina Majorino', 'Keith Carradine', 'Chelsea ...",['George T. Miller'],"['Harry Goodridge', 'Lew Dietz']",[],['Donald Kushner'],[],4419809,"In 1962 Toni Whitney, a nine-year-old girl, an...",1994.0


In [480]:
metadata_df.columns

Index(['index', 'belongs_to_collection', 'budget', 'genres', 'id', 'imdbId',
       'original_language', 'overview', 'popularity', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'director', 'writer', 'producer',
       'executive producer', 'score', 'wikiId', 'Summary', 'release_year'],
      dtype='object')

In [516]:
m = 'The Godfather'
imdb_id = metadata_df[metadata_df['title']==m]['imdbId'].astype(int).to_string(index=False).rjust(7,'0')
imdb_id
# pd.set_option('display.max_colwidth', 50)
# metadata_df[metadata_df['title']==m]['vote_average'].to_string().split()[1]
# director = metadata_df[metadata_df['title']==m]['director'].to_string(index=False)
# ", ".join(ast.literal_eval(metadata_df[metadata_df['title']==m]['director'].to_string(index=False)))

'0068646'

In [255]:
G = pickle.load(open('../dataset/graph.txt','rb'))

In [256]:
#gets the shortest paths from above network given two movie title inputs to create cast_score

def cast_paths(movie1, movie2):
    #shortest_path = nx.shortest_path_length(G, source=movie1, target=movie2)
    cast_score = dict.fromkeys(metadata_df['title'], 0)
    for path in nx.all_simple_paths(G, source=movie1, target=movie2, cutoff = 2):#shortest_path):
        cast_score[path[0]] += 2
    for path in nx.all_simple_paths(G, source=movie1, target=movie2, cutoff = 3):
        for movie in path:
            if cast_score[movie] == 0:
                cast_score[movie] += 1
                
    cast_score[movie1] = 0
    cast_score[movie2] = 0
    
    return cast_score

### Save Cosine Similarity File

In [424]:
#get cosine similarity matrix of vectorized summaries

tfidf_summary = TfidfVectorizer(stop_words='english').fit_transform(metadata_df['Summary'])
cosine_similarity = linear_kernel(tfidf_summary, tfidf_summary)

In [425]:
cosine_similarity

array([[1.        , 0.00991408, 0.00790721, ..., 0.00693665, 0.00551457,
        0.00379024],
       [0.00991408, 1.        , 0.01320353, ..., 0.00132788, 0.00375732,
        0.00685326],
       [0.00790721, 0.01320353, 1.        , ..., 0.00290051, 0.00537821,
        0.00620237],
       ...,
       [0.00693665, 0.00132788, 0.00290051, ..., 1.        , 0.        ,
        0.00408956],
       [0.00551457, 0.00375732, 0.00537821, ..., 0.        , 1.        ,
        0.0028997 ],
       [0.00379024, 0.00685326, 0.00620237, ..., 0.00408956, 0.0028997 ,
        1.        ]])

In [291]:
# import numpy as np
# import h5py

In [292]:
# hf = h5py.File('data.h5', 'w')
# hf.create_dataset('cos_sim', data=cosine_similarity)
# hf.close()

In [295]:
# hfr = h5py.File('data.h5', 'r')
# a = hfr.get('cos_sim')
# np.array(a)

In [ ]:
#### saving array is too expensive in terms of storage space!

### add funcs

In [426]:
#get all cosine similarities for one movie. helper func for cos_sim_match
def get_cos_sim(title, cos_sim = cosine_similarity):
    index = metadata_df.index[metadata_df['title'] == title]
    cs = cosine_similarity[index][0]
    cs[index] = 0 #set cosine sim with self to zero so we don't recommend same movie as input
    return cs

#given values and assuming keys are movie title, create dictionary. helper func for cos_sim_match
def list_to_dict(values):
    keys = metadata_df['title']
    dictionary = dict(zip(keys, values))
    #dictionary = sorted(dictionary.items(), key=lambda item: item[1], reverse=True)
    return dictionary

#get cosine_sim for both movie inputs and add together for cosine_sim score
def cos_sim_match(movie1, movie2):
    cs_movie1 = get_cos_sim(title = movie1)
    cs_movie2 = get_cos_sim(title = movie2)
    summary_score = list_to_dict(cs_movie1*np.max(cs_movie2) + cs_movie2*np.max(cs_movie1))
    return dict(summary_score)

In [427]:
all_genres = {'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western'}

In [428]:
# genres_dict = {}
# for genre in all_genres:
#     genres_dict[genre] = dict.fromkeys(all_genres, 1)
#     for i in metadata_df['genres']:
#         i = lst = ast.literal_eval(i)
#         if genre in i:
#             for j in i:
#                 genres_dict[genre][j] = genres_dict[genre][j]+1
#     genres_dict[genre] = sorted(genres_dict[genre].items(), key=lambda item: item[1], reverse=True)
# genres_dict['Drama']

In [429]:
# genres_df = pd.DataFrame.from_dict(genres_dict)

In [430]:
# genres_df.to_csv('../dataset/genres.csv',index=False)

In [431]:
read_dict = pd.read_csv('../dataset/genres.csv').to_dict('list')
genres_dict = {}
for k, v in read_dict.items():
    genres_dict[k] = [ast.literal_eval(x) for x in v]
genres_dict['Drama']

[('Drama', 10383),
 ('Romance', 2687),
 ('Comedy', 2186),
 ('Thriller', 2093),
 ('Crime', 1571),
 ('Action', 1423),
 ('Mystery', 724),
 ('Adventure', 627),
 ('War', 564),
 ('History', 553),
 ('Family', 496),
 ('Horror', 469),
 ('Foreign', 447),
 ('Fantasy', 392),
 ('Music', 359),
 ('Science Fiction', 327),
 ('Western', 179),
 ('TV Movie', 165),
 ('Animation', 92),
 ('Documentary', 44)]

In [432]:
#create genre score

#randomly pick genre from available options if movie has more than one genre,
#and take the genre most commonly associated with that chosen genre (do for both movies)
#and check to see if those two new genres match each other or any of the original genres
#helper func for match_genres
def randomized_genre_pick(genre1, genre2):
    genre1_pick = genres_dict[random.choice(genre1[0])][1][0]
    genre2_pick = genres_dict[random.choice(genre2[0])][1][0]
    return set([genre1_pick, genre2_pick, genre1, genre2])


def match_genres(movie1, movie2):
    genre1 = list(metadata_df.loc[metadata_df['title'] == movie1, 'genres'])
    genre2 = list(metadata_df.loc[metadata_df['title'] == movie2, 'genres'])
    intersect = set(genre1[0]) & set(genre2[0])

    #if the two movie inputs don't have a genre in common,
    #we'll use randomized_genre_pick to try to find a similar genre in common
    emergency_count = 0
    while len(intersect) == 0:
        intersect = randomized_genre_pick(genre1, genre2)
        emergency_count += 1
        
        #the most common genre is drama. If it can't find a matching one
        #after x iterations, we'll just pick drama as the matching one
        if emergency_count > 20:
            intersect = set(['Drama'])
            break
        
    genre_score = dict.fromkeys(metadata_df['title'], 1)
    for i in metadata_df.index:
        if intersect.issubset(metadata_df['genres'].iloc[i]):
            genre_score[metadata_df['title'].iloc[i]] = 2
    
    #to help avoid picking input movies for output
    genre_score[movie1] = 0
    genre_score[movie2] = 0
    
    return genre_score

#match_genres('Jumanji', 'Grumpier Old Men')

In [433]:
#create score for ranking varaiables like vote and popularity
#find range between two input movies and give higher score to new movies in that range

def range_score(movie1, movie2, prop):
    movie1 = float(metadata_df[metadata_df['title'] == movie1][prop])
    movie2 = float(metadata_df[metadata_df['title'] == movie2][prop])
    
    max_score = max(movie1, movie2)
    min_score = min(movie1, movie2)
    
    score = dict.fromkeys(metadata_df['title'], 1)
    for index in metadata_df.index:
        this_move_score = float(metadata_df[prop].iloc[index])
        if this_move_score > min_score and this_move_score < max_score:
            score[metadata_df['title'].iloc[index]] = 2
    
    score[movie1] = 0
    score[movie2] = 0 
    
    return score

In [434]:
#use levenshtein distance to match mispelled inputs with a movie title in metadata_df

import Levenshtein as lev

def fuzzy_match(movie):
    ratio_list = []
    for i in metadata_df['title']:
        #if statement to remove release years added to titles where name refers to more than one movie
        if i[-1] == ')' and i[-6] == '(':
            ratio_list.append(lev.ratio(movie.lower(), i[0:-6].lower()))
        else:
            ratio_list.append(lev.ratio(movie.lower(), i.lower()))
    return movie, metadata_df['title'].iloc[np.argmax(ratio_list)] #return closest match

In [435]:
#combine scores defined above with weights to calculate score for each movie to recommend

def movie_matcher(movie1, movie2):
    
    #match user query to movie title in metadata_df
    user_query_1, movie1 = fuzzy_match(movie1)
    user_query_2, movie2 = fuzzy_match(movie2)

    #get each score for movie combo
    summary_score = cos_sim_match(movie1, movie2)
    cast_score = cast_paths(movie1, movie2)
    genre_score = match_genres(movie1, movie2)
    vote_score = range_score(movie1, movie2, 'vote_average')
    popularity_score = range_score(movie1, movie2, 'popularity')
    
    #weights for each score
    w1 = 3
    w2 = 1
    w3 = 3
    w4 = 1
    w5 = 1

    #update scores with weights
    summary_score.update((x,y*w1) for x,y in summary_score.items())
    cast_score.update((x,y*w2) for x,y in cast_score.items())
    genre_score.update((x,y*w3) for x,y in genre_score.items())
    vote_score.update((x,y*w4) for x,y in vote_score.items())
    popularity_score.update((x,y*w5) for x,y in popularity_score.items())

    final_score = Counter(summary_score) + Counter(cast_score) + Counter(genre_score) \
                    + Counter(vote_score) + Counter(popularity_score)
    final_score = sorted(final_score.items(), key=lambda item: item[1], reverse=True)

    print('User Query 1: '+user_query_1+'    Matched to: '+movie1)
    print('User Query 2: '+user_query_2+'    Matched to: '+movie2)
    
    return final_score[0:10]#summary_score, cast_score, genre_score

In [436]:
movie_matcher('2001 a psace oddysyey', 'Blad Runer')

User Query 1: 2001 a psace oddysyey    Matched to: 2001: A Space Odyssey
User Query 2: Blad Runer    Matched to: Blade Runner


[('The Fifth Element', 10.034464744959557),
 ('Alien', 9.046067273606106),
 ('The Tommyknockers', 9.042524574842103),
 ('Rise of the Planet of the Apes', 9.039393155622776),
 ('The Day the Earth Stood Still(2008)', 9.032847616752592),
 ('Superman(1978)', 9.032319582475207),
 ('Saturn 3', 9.030049787382612),
 ('Cowboys & Aliens', 9.02878006539675),
 ('Colossus: The Forbin Project', 9.027725056432946),
 ('The Star Wars Holiday Special', 9.024792166662813)]

In [438]:
res = movie_matcher('Titanic','Harry Potter')

User Query 1: Titanic    Matched to: Titanic(1997)
User Query 2: Harry Potter    Matched to: Harper


In [440]:
res[0][0]

'Alive(1993)'

In [421]:
def matched_movie_info(movie1, movie2):
    output_lst = movie_matcher(movie1, movie2)
    for movie, score in output_lst:
        metadata = metadata_df[metadata_df['title']==movie]

In [276]:
metadata_df[metadata_df['title']=='Titanic(1997)']

,index,belongs_to_collection,budget,genres,id,imdbId,original_language,overview,popularity,release_date,...,vote_count,cast,director,writer,producer,executive producer,score,wikiId,Summary,release_year
1613,2424,None,200000000,"['Drama', 'Romance', 'Thriller']",597,120338.0,en,"84 years later, a 101-year-old woman named Ros...",26.88907,1997-11-18,...,7770.0,"[Kate Winslet, Leonardo DiCaprio, Frances Fish...",['James Cameron'],['James Cameron'],"['James Cameron', 'Jon Landau']",['Rae Sanchini'],[],52371,"In 1996, treasure hunter Brock Lovett and his...",1997.0
